# Forecasting
Prediction of sales is the central task in this challenge. you want to predict daily sales in various stores up to 6 weeks ahead of time. This will help the company plan ahead of time. 

The following steps outline the various sub tasks needed to effectively do this: 

In [1]:
# importing of libraries
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import ticker
from statsmodels.tsa.stattools import adfuller, acf, pacf
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from pandas.plotting import scatter_matrix
from sklearn.model_selection import cross_val_score
import seaborn as sns
import os,sys
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from timeseries import TimeSeries
from modeling import Modeler
sns.set()


2022-05-25 09:54:13.961953: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-25 09:54:13.961987: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


/home/martin/Documents/pharm_sales/notebooks


2.2 Building models with sklearn pipelines
At this point, all our features are numeric. Since our problem is a regression problem, you can narrow down the list of algorithms you can use for modelling. 

A reasonable starting point will be to use any of the tree based algorithms. Random forests Regressor will make for a good start. 

Also, for the sake of this challenge, work with sklearn pipelines. This makes modeling modular and more reproducible. Working with pipelines will also significantly reduce your workload when you are moving your setup into files for the next part of the challenge. Extra marks will be awarded for doing this.


In [2]:
train_ = pd.read_csv("../data/cleaned_train.csv")
test = pd.read_csv("../data/cleaned_test.csv")
train_.drop(['DayOfWeek','DayOfYear','WeekOfYear',
            'Customers',"Month","Day"],axis=1,inplace=True)
test.drop(["Id",'DayOfWeek','DayOfYear','WeekOfYear'
           ,"Month","Day"],axis=1,inplace=True)
train=train_.loc[:,train_.columns!='Sales']
train['Sales']=train_['Sales']
train.sort_values(["Year"], ascending=False ,ignore_index=True, inplace=True)
test.sort_values(["Year"], ascending=False ,ignore_index=True, inplace=True)
train.index.name = 'Year'
train = train.set_index('Year')
test.index.name = 'Year'
test = test.set_index('Year')


In [3]:
train.head()

,Assortment,AvgCustomers,AvgSales,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,DailyCustomers,DailySales,Open,Promo,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,SchoolHoliday,StateHoliday,Store,StoreType,Sales
Year,,,,,,,,,,,,,,,,,,,
2015,0.0,0.183768,0.157115,0.019521,0.909091,0.930435,0.209999,0.180225,1.0,1.0,0.0,0.255319,0.166667,0.0,1.0,0.333333,0.000000,0.0,0.242764
2015,0.0,0.235718,0.206591,0.015470,0.909091,0.991304,0.269365,0.236980,1.0,0.0,0.0,0.638298,0.666667,1.0,0.0,0.333333,0.350766,0.0,0.207454
2015,0.0,0.395239,0.462220,0.064825,0.909091,0.921739,0.451656,0.530211,1.0,0.0,0.0,0.085106,0.666667,1.0,0.0,0.333333,0.341749,0.0,0.331799
2015,0.0,0.626386,0.874092,0.081031,0.363636,0.982609,0.713926,1.000000,1.0,0.0,0.0,0.191489,0.833333,0.5,0.0,0.333333,0.340848,0.0,0.409744
2015,1.0,0.400381,0.375524,0.002210,0.454545,0.956522,0.457531,0.430761,1.0,0.0,0.0,0.361702,0.166667,1.0,0.0,0.333333,0.338142,0.0,0.294127


In [4]:
inputs = train[:1]

# Modeling with RandomForest

In [5]:
analyzer = Modeler(train)

In [6]:
forecast = analyzer.regr_models(inputs,RandomForestRegressor,column='Sales',
                                encoded=True,n_estimators=10)

/home/martin/Documents/pharm_sales/env/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [7]:
scores,forecast = forecast

In [9]:
scores

0.007165473383690862

# Modeling with LSTM

Unit Root Tests helps us determine the stationarity of our data

In [3]:
timeseries = TimeSeries(train)
# timeseries.perform_adfuller('Sales')

2022-05-25 04:58:36,870:logger:Initialized the time series class


Remove stationarity from the data

In [10]:
train['Sales'] = timeseries.remove_stationarity(train.Sales.values)

In [9]:
SIZE = len(train.Sales)
WINDOW_SIZE = 489
BATCH_SIZE= SIZE-WINDOW_SIZE*2
EPOCHS = 5
ID = test['Id']

In [10]:
DatasetTrain = timeseries.split_dataset(train.Sales[0:BATCH_SIZE])
DatasetVal = timeseries.split_dataset(train.Sales[BATCH_SIZE])

2022-05-25 04:17:45,860:logger:Successfully windowed the dataset
2022-05-25 04:17:45,928:logger:Successfully windowed the dataset


In [11]:
model = timeseries.model()
results = model.fit(DatasetTrain, epochs=EPOCHS, validation_data=DatasetVal, verbose=1)

2022-05-25 04:17:47,740:logger:Successfully modeled the neural network


Epoch 1/10
1617/1617 [==============================] - 113s 67ms/step - loss: 0.0041
Epoch 2/10
1617/1617 [==============================] - 113s 70ms/step - loss: 0.0037
Epoch 3/10
1617/1617 [==============================] - 123s 76ms/step - loss: 0.0037
Epoch 4/10
1617/1617 [==============================] - 124s 76ms/step - loss: 0.0037
Epoch 5/10
1617/1617 [==============================] - 109s 68ms/step - loss: 0.0037
Epoch 6/10
1617/1617 [==============================] - 112s 69ms/step - loss: 0.0037
Epoch 7/10
1617/1617 [==============================] - 108s 67ms/step - loss: 0.0037
Epoch 8/10
1617/1617 [==============================] - 108s 67ms/step - loss: 0.0037
Epoch 9/10
1617/1617 [==============================] - 101s 63ms/step - loss: 0.0037
Epoch 10/10
1617/1617 [==============================] - 104s 65ms/step - loss: 0.0037


In [ ]:
forecast = timeseries.model_forecast(model, train.Sales.values[:, np.newaxis], WINDOW_SIZE,SIZE=BATCH_SIZE)
results = forecast[BATCH_SIZE-WINDOW_SIZE:-1]
results_ = scaler.inverse_transform(results.reshape(-1,1))
XValid_ = scaler.inverse_transform(XValid.reshape(-1,1))

2022-05-25 04:42:53.952693: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1264737864 exceeds 10% of free system memory.


In [ ]:
view_forecast_= timeseries.view_forecast(DateValid,XValid_,results_,results,WINDOW_SIZE=48)